In [172]:
#!pip install deap 

In [173]:
import numpy as np
import math
from deap import base, creator, tools
import random
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize
import json
import pandas as pd


In [174]:
from typing import List, Optional
from pydantic import BaseModel



class ClassIndividual(BaseModel):
    index: int
    var_decicison: List[float]
    Fitness: Optional[float]
    RCE: Optional[str]
    diversidade: float

class SetupInd:
    def __init__(self, params, dadosEntrada=None):

        #! Parametros JSON
        self.params = params
        self.DADOS_ENTRADA = dadosEntrada  # Dados da entrada do usuario

        self.CXPB = params["CROSSOVER"]
        self.MUTPB = params["MUTACAO"]
        self.NGEN = params["NUM_GENERATIONS"]
        self.POP_SIZE = params["POP_SIZE"]
        self.SIZE_INDIVIDUAL = params["IND_SIZE"]
        self.TAXA_GENERATION = params["RCE_REPOPULATION_GENERATIONS"]
        self.CROSSOVER, self.MUTACAO, self.NUM_GENERATIONS, self.POPULATION_SIZE = (
            self.CXPB,
            self.MUTPB,
            self.NGEN,
            self.POP_SIZE,
        )

        self.delta = params["DELTA"]

        #! Parâmetros do algoritmo de Rastrigin
        self.evaluations = 0
        self.num_repopulation = int(self.NUM_GENERATIONS * self.TAXA_GENERATION)
        self.type = params["type"].lower()
        if self.type == "maximize":
            print("Método escolhido: Maximizar")
            creator.create("Fitness", base.Fitness, weights=(1.0,))
        else:
            print("Método escolhido: Minimizar")
            creator.create("Fitness", base.Fitness, weights=(-1.0,))

        self.dataset = {}
        self.beta_constants = {}

        #!Criando individuo pelo deap com seus atributos
        self.toolbox = base.Toolbox()

        # Função de fitness de minimização
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

        # parâmetros
        creator.create(
            "Individual", list, fitness=creator.FitnessMin, rce=str, index=int
        )

        # Variavel de decisão
        self.toolbox.register("attribute", random.uniform, -5.12, 5.12)

        # registrando os individuos
        self.toolbox.register(
            "individual",
            tools.initRepeat,
            creator.Individual,
            self.toolbox.attribute,
            n=self.SIZE_INDIVIDUAL,
        )

        #! individuos com dados de entrada
        if self.DADOS_ENTRADA:
            creator.create(
                "FrameworkIndividual",
                list,
                fitness=creator.FitnessMin,
                tipo=self.DADOS_ENTRADA[0],
                rce=str,
            )
            self.toolbox.register(
                "varDecision",
                random.uniform,
                self.DADOS_ENTRADA[2][0],
                self.DADOS_ENTRADA[2][1],
            )
            self.toolbox.register(
                "newIndividual",
                tools.initRepeat,
                creator.FrameworkIndividual,
                self.toolbox.varDecision,
                n=self.DADOS_ENTRADA[1],
            )

            self.toolbox.register(
                "FrameworkPopulation",
                tools.initRepeat,
                list,
                self.toolbox.newIndividual,
            )



        self.toolbox.register("mate", tools.cxTwoPoint)
        self.toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
        self.toolbox.register("select", tools.selTournament, tournsize=3)
        self.toolbox.register("evaluate", self.calculaFitness)
                #! paramentos evolutivos
        self.toolbox.register(
            "population", tools.initRepeat, list, self.toolbox.individual
        )


    def calculaFitness(self, individual):
            if self.type == "minimaze":
                result = minimize(
                    self.rastrigin, x0=np.zeros(self.SIZE_INDIVIDUAL), method="BFGS"
                )
                fitness_value = result.fun
            return fitness_value

    def rastrigin(self, individual):
            self.evaluations += 1
            rastrigin = 10 * self.SIZE_INDIVIDUAL
            for i in range(self.SIZE_INDIVIDUAL):
                rastrigin += individual[i] * individual[i] - 10 * (
                    math.cos(2 * np.pi * individual[i])
                )
            return rastrigin

    def create_population(self):
        new_pop = self.toolbox.population(
            n=self.POP_SIZE
        )
        return new_pop



def load_params(file_path):
    with open(file_path, "r") as file:
        params = json.load(file)
    return params



**Parameters**:
*name* – The name of the class to create.

*base* – A base class from which to inherit.

*attribute* – One or more attributes to add on instantiation of this class, optional.




This above line is exactly the same as defining in the creator module something like the following.

```
create("Foo", list, bar=dict, spam=1)

class Foo(list):
    spam = 1

    def __init__(self):
        self.bar = dict()
```

In [175]:
import easygui

class IMCCalculator:
    def __init__(self):
        self.window_title = "Calculadora de IMC"
        self.text = "Insira seus dados abaixo:"
        self.weight_label = "Peso (kg):"
        self.height_label = "Altura (m):"

    def get_input(self):
        weight = easygui.integerbox(self.weight_label, self.window_title)
        height = easygui.integerbox(self.height_label, self.window_title)
        return weight, height

    def calculate_imc(self, weight, height):
        imc = weight / (height * height)
        return imc

    def display_result(self, imc):
        message = "Seu IMC é: {:.2f}".format(imc)
        easygui.msgbox(message, self.window_title)

def main():
    calculator = IMCCalculator()
    weight, height = calculator.get_input()
    imc = calculator.calculate_imc(weight, height)
    calculator.display_result(imc)

#main()

In [201]:

params = load_params(r"C:\Users\rpped\Documents\Estagio - Pedro V - Usinagem\Code\assets\parameters.json")

new_setup = SetupInd(params)

pop = new_setup.create_population()


db_ind: List[ClassIndividual] = []

for i, ind in enumerate(pop):
    print(i, ind)
    print(type(ind))
    print("\n")

    # Create ClassIndividual instance
    individual = ClassIndividual(
        index=i,
        var_decicison=ind,
        diversidade = sum(ind),
        Fitness = (10.0),
        RCE= "rce"
    )

    db_ind.append(individual)

    if (i == 5):
        break

print("tipo ind =",type(db_ind[0].var_decicison))

Método escolhido: Minimizar
0 [-2.3041849680663846, -3.072405849195493, 4.3572680881834644, 0.9978998906625183, -3.597891479461986, 1.5532656532540345, 4.308183736061575, -1.1606876872072132, -3.640485217681414, -1.2653417971127543]
<class 'deap.creator.Individual'>


1 [-1.2187576996131941, -1.9657714125834262, -0.9865235058713662, 3.246720314962361, 4.597255848841553, -3.1255464888330335, -3.670519929751288, -1.9544202060237037, 0.49625138026433646, -3.176116614710494]
<class 'deap.creator.Individual'>


2 [1.207334892381203, 0.17330239224831168, 1.0166908484945898, -4.3668560440247, -0.5321379211046198, 0.33550530825658154, -4.190796649471389, 1.35631333389387, -2.4461109030788966, 2.8064424532091348]
<class 'deap.creator.Individual'>


3 [0.4952604592423304, -0.6767662984461849, 4.672376415756156, 2.6698376822562535, -0.05556257977629464, -4.361382462652321, 0.8403378076895773, -2.408947092889448, -3.5177604962920532, 3.3833175605574537]
<class 'deap.creator.Individual'>


4 [1.597

## Setup

In [177]:
import numpy as np
import math
from deap import base, creator, tools
import random
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize
import json
import pandas as pd


class Setup:
    def __init__(self, params, dadosEntrada=None):

        #! Parametros JSON
        self.params = params
        self.DADOS_ENTRADA = dadosEntrada  # Dados da entrada do usuario

        self.CXPB = params["CROSSOVER"]
        self.MUTPB = params["MUTACAO"]
        self.NGEN = params["NUM_GENERATIONS"]
        self.POP_SIZE = params["POP_SIZE"]
        self.SIZE_INDIVIDUAL = params["IND_SIZE"]
        self.TAXA_GENERATION = params["RCE_REPOPULATION_GENERATIONS"]
        self.CROSSOVER, self.MUTACAO, self.NUM_GENERATIONS, self.POPULATION_SIZE = (
            self.CXPB,
            self.MUTPB,
            self.NGEN,
            self.POP_SIZE,
        )

        self.delta = params["DELTA"]

        #! Parâmetros do algoritmo de Rastrigin
        self.evaluations = 0
        self.num_repopulation = int(self.NUM_GENERATIONS * self.TAXA_GENERATION)
        self.type = params["type"].lower()
        if self.type == "maximize":
            print("Método escolhido: Maximizar")
            creator.create("Fitness", base.Fitness, weights=(1.0,))
        else:
            print("Método escolhido: Minimizar")
            creator.create("Fitness", base.Fitness, weights=(-1.0,))

        self.dataset = {}
        self.beta_constants = {}

        #!Criando individuo pelo deap com seus atributos
        self.toolbox = base.Toolbox()

        # Função de fitness de minimização
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

        # parâmetros
        creator.create(
            "Individual", list, fitness=creator.FitnessMin, rce=str, index=int
        )

        # Variavel de decisão
        self.toolbox.register("attribute", random.uniform, -5.12, 5.12)

        # registrando os individuos
        self.toolbox.register(
            "individual",
            tools.initRepeat,
            creator.Individual,
            self.toolbox.attribute,
            n=self.SIZE_INDIVIDUAL,
        )

        #! individuos com dados de entrada
        if self.DADOS_ENTRADA:
            creator.create(
                "FrameworkIndividual",
                list,
                fitness=creator.FitnessMin,
                tipo=self.DADOS_ENTRADA[0],
                rce=str,
            )
            self.toolbox.register(
                "varDecision",
                random.uniform,
                self.DADOS_ENTRADA[2][0],
                self.DADOS_ENTRADA[2][1],
            )
            self.toolbox.register(
                "newIndividual",
                tools.initRepeat,
                creator.FrameworkIndividual,
                self.toolbox.varDecision,
                n=self.DADOS_ENTRADA[1],
            )

            self.toolbox.register(
                "FrameworkPopulation",
                tools.initRepeat,
                list,
                self.toolbox.newIndividual,
            )

        #! paramentos evolutivos
        self.toolbox.register(
            "population", tools.initRepeat, list, self.toolbox.individual
        )

        self.toolbox.register("mate", tools.cxTwoPoint)
        self.toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
        self.toolbox.register("select", tools.selTournament, tournsize=3)
        self.toolbox.register("evaluate", self.evaluate_fitness)

    def evaluate_fitness(self, individual):
        if self.type == "minimaze":
            result = minimize(
                self.rastrigin, x0=np.zeros(self.SIZE_INDIVIDUAL), method="BFGS"
            )
            fitness_value = result.fun
        return fitness_value

    def gerarDataset(self, excel):
        df = pd.read_excel(excel)
        print(df.columns)
        self.dataset = {
            "CXPB": self.CROSSOVER,
            "TAXA_MUTACAO": self.MUTACAO,
            "NUM_GEN": self.NUM_GENERATIONS,
            "POP_SIZE": self.POPULATION_SIZE,
            "IND_SIZE": self.SIZE_INDIVIDUAL,
            "evaluations": self.evaluations,
            "NUM_REPOPULATION": self.num_repopulation,
        }

    def rastrigin(self, individual):
        self.evaluations += 1
        rastrigin = 10 * self.SIZE_INDIVIDUAL
        for i in range(self.SIZE_INDIVIDUAL):
            rastrigin += individual[i] * individual[i] - 10 * (
                math.cos(2 * np.pi * individual[i])
            )
        return rastrigin

    def rastrigin_decisionVariables(self, individual, decision_variables):
        self.evaluations += 1
        rastrigin = 10 * len(decision_variables)
        for i in range(len(decision_variables)):
            rastrigin += individual[i] * individual[i] - 10 * (
                math.cos(2 * np.pi * individual[i])
            )
        return rastrigin

    def rosenbrock(self, x):
        return np.sum(100 * (x[1:] - x[:-1] ** 2) ** 2 + (1 - x[:-1]) ** 2)

    def globalSolutions(self):
        n_dimensions = 2

        try:
            rastrigin_result = minimize(
                self.rastrigin, x0=np.zeros(n_dimensions), method="BFGS"
            )
            rastrigin_minimum = rastrigin_result.fun
            rastrigin_solution = rastrigin_result.x

            rosenbrock_result = minimize(
                self.rosenbrock, x0=np.zeros(n_dimensions), method="BFGS"
            )
            rosenbrock_minimum = rosenbrock_result.fun
            rosenbrock_solution = rosenbrock_result.x

        except Exception as e:
            print("Erro ao tentar encontrar o ótimo global das funções: ", e)

        print("\n\nÓtimo global da função Rastrigin: ", rastrigin_minimum)
        print("Solução: ", rastrigin_solution)
        print()
        print("Ótimo global da função Rosenbrock: ", rosenbrock_minimum)
        print("Solução: ", rosenbrock_solution)

## COLOCANDO PARAMETROS

-----------------------
def  Critério_1
Aptidão do indivíduo é menor ou igual que (1 + parametro_perc) * aptidão do melhor indivíduo

Para todos os indivíduos da pop atual
Se individuo atende ao critério 1
     inserir HOF
Se lista vazia
          inserir individuo
     Senão
          Se individuo é diferente de todos da lista
                inserir individuo
-----------------------
def compararIndividuosSelecionados():
Para todos indivíduos da lista
       def Calcular Diff - diferença entre as variáveis de indivíduo e individuo lista
       Se Diff não é maior que parâmetro_diff em mais de parâmetro_var -> numero de variaveis diferentes (min 2)
             Retorna falso
Fim Para
Retorna verdadeiro



In [178]:
from deap import base, creator

def load_params(file_path):
    with open(file_path, "r") as file:
        params = json.load(file)
    return params

params = load_params(r"C:\Users\rpped\Documents\Estagio - Pedro V - Usinagem\Code\assets\parameters.json")

# Criando uma instância da classe Setup
setup = Setup(params)


#! Carregar os dados
turbina = pd.read_csv(r'C:\Users\rpped\Documents\Estagio - Pedro V - Usinagem\Code\assets\T1.csv')
turbina.columns = ['Data/hora', 'ActivePower(kW)', 'WindSpeed(m/s)', 'Curva_Teórica(KWh)',"Direção do Vento"]
display(turbina.head())

X = turbina[['WindSpeed(m/s)']].values
y = turbina[['ActivePower(kW)']].values
data_hora = turbina[['Data/hora']].values
curva = turbina[['Curva_Teórica(KWh)']].values
vento = turbina[['Direção do Vento']].values


Método escolhido: Minimizar


,Data/hora,ActivePower(kW),WindSpeed(m/s),Curva_Teórica(KWh),Direção do Vento
0,01 01 2018 00:00,380.047791,5.311336,416.328908,259.994904
1,01 01 2018 00:10,453.769196,5.672167,519.917511,268.641113
2,01 01 2018 00:20,306.376587,5.216037,390.900016,272.564789
3,01 01 2018 00:30,419.645905,5.659674,516.127569,271.258087
4,01 01 2018 00:40,380.650696,5.577941,491.702972,265.674286


## Algoritimo Evolutivo

In [179]:
import numpy as np
import math
from deap import base, creator, tools
import random
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize
import json
import pandas as pd
import numpy as np
import math
from deap import base, creator, tools
import random
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize
import json
import pandas as pd


class AlgoritimoEvolutivoRCE:
    def __init__(self, setup):
        self.setup = setup
        self.stats = tools.Statistics(key=lambda ind: ind.fitness.values)
        self.stats.register("avg", np.mean)
        self.stats.register("std", np.std)
        self.stats.register("min", np.min)
        self.stats.register("max", np.max)

        self.logbook = tools.Logbook()
        self.hof = tools.HallOfFame(1)
        self.POPULATION = self.setup.toolbox.population(n=self.setup.POP_SIZE)
        self.hof.update(self.POPULATION)

        if self.setup.DADOS_ENTRADA:
            self.POP_OPTIMIZATION = self.setup.toolbox.population(n=self.setup.POP_SIZE)

        self.pop_RCE = []
        self.best_solutions_array = []
        self.best_individual_array = []
        self.allIndividualValuesArray = []
        self.data = {}
        self.repopulation_counter = 0
        self.allFitnessValues = {}
        self.validateCounter = 0
        self.CONJUNTO_ELITE_RCE = set()

    def pesquisarLinear(self, array, num, pos=0):
        while (pos < len(array)) and (array[pos] != num):
            pos += 1

        if array[pos] == num:
            print("Valor encontrado")

    def registrarDados(self, generation):

        # Registrar estatísticas e melhores soluções
        for ind in self.POPULATION:
            avg_fitness_per_generation = np.mean(ind.fitness.values)
            std_deviation = np.std(ind.fitness.values)

        #! PEgandos os dados e colocando no df
        self.data = {
            "Generations": generation + 2,
            "Variaveis de Decisão": self.hof[0],
            "Evaluations": self.setup.evaluations,
            "Ind Valido": self.hof[0].fitness.valid,
            "Best Fitness": self.hof[0].fitness.values,
            "Media": avg_fitness_per_generation,
            "Desvio Padrao": std_deviation,
        }

        self.best_individual_array.append(self.data)

        self.visualizarPopAtual(generation, [avg_fitness_per_generation, std_deviation])

    def checkClonesInPop(self, ind, new_pop):
        is_clone = False
        for other_ind in new_pop:
            if (
                ind == other_ind
                and sum(ind) == sum(other_ind)
                and ind.index != other_ind.index
            ):
                is_clone = True
                break
        return is_clone

    def generateInfoIndividual(self, new_pop, generation):
        ind_array = []

        for i, ind in enumerate(new_pop):
            # print(f"Index[{ind.index}] - ind_variables {ind} \n Fitness = {ind.fitness.values} ")

            ind.index = i

            ind_info = {
                "Generations": generation,
                "index": ind.index,
                "Variaveis de Decisão": ind,
                "Fitness": ind.fitness.values[0],
                "RCE": ind.rce,
                "Diversidade": np.sum(ind),
            }

            # Adicionar a informação de clone ao dicionário
            #is_clone = self.checkClonesInPop(ind, new_pop)
            #ind_info["CLONE"] = "SIM" if is_clone else "NAO"

            ind_array.append(ind_info)

        return ind_array

    def show_ind_df(self, array, text):
        df = pd.DataFrame(array)
        print(text)
        display(df.head(40))

        # contar quantos SIM na coluna CLONE se a coluna RCE for SIM
        #display(df[df["RCE"] != ""]["CLONE"].value_counts())

    def criterio1(self, new_pop, porcentagem, k=30):
        """Seleciona os candidatos ao conjunto elite com base nas diferenças percentuais de aptidão."""
        self.cout(f"CRITÉRIO 1 RCE - Selecionando candidatos ao conjunto elite")
        elite_individuals = []

        # Ordenar a população em ordem crescente de aptidão e selecionar os k primeiros indivíduos
        sorted_population = sorted(self.POPULATION, key=lambda x: x.fitness.values[0])

        # Obter o melhor indivíduo (HOF) da população
        best_ind = new_pop[0]
        best_fitness = best_ind.fitness.values[0]
        max_difference = (1 + porcentagem) * best_fitness

        # Selecionar com as menores diferenças percentuais
        for ind in sorted_population:
            if ind.fitness.values[0] <= max_difference:
                elite_individuals.append(ind)
            else:
                break  # Parar a seleção quando a diferença percentual for maior que o limite

        # Colocando na pop aleatória
        print(
            f"Calculando percentual de {porcentagem*100}% com base no melhor fitness = {best_fitness} e pegando os {len(elite_individuals)} melhores.\n Porcentagem de {best_fitness} = {max_difference} "
        )
        for i, ind in enumerate(elite_individuals):
            new_pop[i] = self.setup.toolbox.clone(ind)
            new_pop[i].rce = "SIM_1"

        return elite_individuals

    def criterio2_alternative(self, ind_selecionados, delta=6):
        """Comparar as variáveis de decisão de cada indivíduo e verificar se existem diferenças superiores a 'delta'."""
        self.cout(
            f"CRITÉRIO 2 - Comparar as variáveis de decisão de cada indivíduo e verificar valores superiores a 'delta' = {delta}."
        )
        self.CONJUNTO_ELITE_RCE.clear()

        for i in range(len(ind_selecionados)):

            #    todo   Calcular Diff - diferença entre as variáveis de indivíduo e individuo lista

            diff = np.array(ind_selecionados[i]) - np.array(ind_selecionados[0])

            # todo retornar true ou false caso ind seja diferente para colocar no array correto
            if sum(diff) > delta:  # ind diferente
                if (ind_selecionados[i] not in self.pop_RCE) and (
                    ind_selecionados[i] not in ind_selecionados[0]
                ):
                    self.pop_RCE.append(self.POP_OPTIMIZATION[i])
                    self.CONJUNTO_ELITE_RCE.add(tuple(self.POP_OPTIMIZATION[i]))
                    ##print("Delta= ", sum(diff))

        if not self.pop_RCE:
            print("Nenhum indivíduo atende aos critérios. :( ")

        print("Tamanho Elite = ", len(self.pop_RCE))
        print("Tamanho Elite = ", len(self.CONJUNTO_ELITE_RCE))

        return self.pop_RCE

    def newCriterio(self, population, delta):
        self.cout(f"CRITÉRIO 2 RCE ")
        self.CONJUNTO_ELITE_RCE.clear()
        self.pop_RCE = []

        def criterio1_reduzido(population, porcentagem=0.40):
            best_ind = self.elitismoSimples(population)[0]
            best_fitness = best_ind.fitness.values[0]
            max_difference = (1 + porcentagem) * best_fitness
            print(f"Fitness calculado com {porcentagem} = {round(max_difference)}")
            return max_difference, best_ind

        max_difference, best_ind = criterio1_reduzido(population)
        self.pop_RCE.append(best_ind)

        # todo colocar hof no rce e ajustar parâmetros sim e nao
        def calculaDiff(ind, lista, valor_limite):
            count_ind = 0
            count = 0
            
            for i in range(len(lista)):
                diff = abs(np.array(ind) - np.array(lista[i]))
                array = list(diff)
                
                if sum(diff) > 0.0:
                    print("array ind selecionados: ", list(diff))
                    print("\n Soma Diff = ", sum(diff))
                    print("contador = ", count)
                    print("contador 2 = ", count_ind)

                    

                    for j in range(len(array)):
                        #print("debug", array[j])

                        if array[j] > delta:
                            count += 1

                        if count >= valor_limite:
                            count_ind += 1
                            # break

                        if j == len(lista) - 1:
                            print("Não é diferente ")
                            # return False
                            break

                    if len(array) >= valor_limite:
                        # if count_ind >= len(array):
                        print("Ind Diferente RCE",array)
                        return True
                    else:
                        return False

                else:
                    return False

        for ind in population:
            # criterio 1
            if ind.fitness.values[0] <= max_difference:
                # criterio 2
                diff = calculaDiff(ind, self.pop_RCE, 3) # delta como valor limite
                if diff:
                    if ind not in self.pop_RCE:
                        self.pop_RCE.append(ind)
                        self.CONJUNTO_ELITE_RCE.add(tuple(ind))

        if len(self.pop_RCE) == 1:
            print("Nenhum indivíduo atende aos critérios. :( ")

        print("Tamanho Elite = ", len(self.pop_RCE))
        print("Tamanho Elite = ", len(self.CONJUNTO_ELITE_RCE))

        return self.pop_RCE

    def aplicar_RCE(self, generation, current_population):

        #! a - Cria uma pop aleatória (eliminando a pop aleatória criada na execução anterior do RCE)
        new_pop = self.setup.toolbox.population(
            n=self.setup.POP_SIZE
        )  # retorna uma pop com lista de individuos de var de decisão

        # Avaliar o fitness da população atual
        self.avaliarFitnessIndividuos(current_population)
        self.calculateFitnessGeneration(current_population)

        #! b - Coloca o elite hof da pop anterior  no topo (0)
        pop = self.elitismoSimples(current_population)
        print(
            f"Elitismo HOF Index[{pop[0].index}] {pop[0]} \n Fitness = {pop[0].fitness.values} | Diversidade = {sum(pop[0])}"
        )  # pop[0] é o melhor individuo HOF
        new_pop[0] = self.setup.toolbox.clone(pop[0])

        #! critério 1 e obtém os N melhores com 30% do valor do melhor fitness
        # ind_selecionados = self.criterio1(self.POPULATION, 0.30, k=30)

        #! Critério 2 usando este array e vai colocando os indivíduos selecionados pelo critério 2 na pop aleatória (passo a)
        # ind_diferentes_var = self.criterio2_alternative(ind_selecionados, delta=7)
        ind_diferentes_var = self.newCriterio(
            current_population, delta=self.setup.delta
        )

        # cOLOCANDO ATRUBUTOS
        for i, ind in enumerate(ind_diferentes_var, start=0):
            new_pop[0].rce = "HOF"
            if i > 0:
                new_pop[i] = self.setup.toolbox.clone(ind)
                new_pop[i].rce = "SIM"

        #! Criterio 3 retorna pop aleatória modificada (com hof + rce + Aleatorio)
        self.calculateFitnessGeneration(new_pop)
        self.cout(f"CRITERIO 3 - População aleatória modificada [HOF,RCE,Aleatorio] ")
        conjunto_elite = self.generateInfoIndividual(new_pop, generation)
        self.show_ind_df(conjunto_elite, "Individuos da nova população aleatória")
        return new_pop

    def elitismoSimples(self, pop):
        self.hof.update(pop)
        pop[0] = self.setup.toolbox.clone(self.hof[0])
        return pop

    def criterio2(self, elite_individuals, delta):
        """Comparar as variáveis de decisão de cada indivíduo e verificar se existem diferenças superiores a 'delta'."""
        self.cout(
            f"CRITÉRIO 2 - Comparar as variáveis de decisão de cada indivíduo e verificar se existem diferenças superiores a 'delta' = {delta}."
        )
        self.pop_RCE = []
        self.CONJUNTO_ELITE_RCE.clear()

        for i in range(len(elite_individuals)):
            current_individual = elite_individuals[i]
            is_diferente = False

            for j in range(i + 1, len(elite_individuals)):
                other_individual = elite_individuals[j]
                diff_counter = 0

                for var_index in range(len(current_individual)):
                    current_var = current_individual[var_index]
                    other_var = other_individual[var_index]

                    if abs(current_var - other_var) > delta:
                        # print(abs(current_var - other_var))
                        diff_counter += 1

                if diff_counter >= 1:
                    is_diferente = True

            if is_diferente:
                # print(f"Indivíduo do tipo {type(current_individual)} VAR({current_individual})\n diferente! adicionado à nova população.")
                self.pop_RCE.append(current_individual)
                self.CONJUNTO_ELITE_RCE.add(tuple(current_individual))

        if not self.pop_RCE:
            print("Nenhum indivíduo atende aos critérios. :( ")

        print("Tamanho Elite = ", len(self.pop_RCE))
        return self.pop_RCE

    def avaliarFitnessIndividuos(self, pop):
        """Avaliar o fitness dos indivíduos da população atual."""
        fitnesses = map(self.setup.toolbox.evaluate, pop)
        for ind, fit in zip(pop, fitnesses):
            if ind.fitness.values:
                ind.fitness.values = [fit]

    def calculateFitnessGeneration(self, new_pop):
        # Calculando o fitness para geração
        for ind in new_pop:
            if not ind.fitness.valid:
                fitness_value = self.setup.toolbox.evaluate(ind)
                ind.fitness.values = (fitness_value,)

    def checkDecisionVariablesAndFitnessFunction(
        self, decision_variables, fitness_function
    ):

        # Verificar se as variáveis de decisão e a função de fitness foram fornecidas
        if decision_variables is None and fitness_function is None:
            # Gerar variáveis de decisão aleatórias para os indivíduos
            decision_variables = [
                random.random() for _ in range(self.setup.SIZE_INDIVIDUAL)
            ]

            # Definir a função de fitness padrão como a função Rastrigin
            fitness_function = self.setup.rastrigin_decisionVariables

        if decision_variables is None or fitness_function is None:
            if not hasattr(self, "decision_variables") or not hasattr(
                self, "fitness_function"
            ):
                raise ValueError(
                    "Variáveis de decisão e função de fitness não definidas. Use set_decision_variables_and_fitness_function primeiro."
                )
        else:
            self.decision_variables = decision_variables
            self.fitness_function = fitness_function

            # Definir a função de fitness com base na função fornecida
            def fitness_func(individual):
                return self.fitness_function(individual, self.decision_variables)

            # Registrar a função de fitness no toolbox
            self.setup.toolbox.register("evaluate", fitness_func)

    #! Main LOOP
    def run(self, RCE=False, decision_variables=None, fitness_function=None, num_pop=0):

        if self.setup.DADOS_ENTRADA:
            population = [self.POPULATION, self.POP_OPTIMIZATION]
        else:
            population = [self.POPULATION]

        # Avaliar o fitness da população inicial
        self.avaliarFitnessIndividuos(population[num_pop])

        # Selecionando as variaveis de decisao e afuncao objeti
        self.checkDecisionVariablesAndFitnessFunction(
            decision_variables, fitness_function
        )

        #! Loop principal através das gerações
        for current_generation in range(self.setup.NGEN):

            # Selecionar os indivíduos para reprodução
            offspring = self.setup.toolbox.select(
                population[num_pop], k=len(population[num_pop])
            )

            # Clone the selected individuals
            offspring = [self.setup.toolbox.clone(ind) for ind in offspring]

            # Aplicar crossover
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < self.setup.CXPB:
                    self.setup.toolbox.mate(child1, child2)
                    del child1.fitness.values
                    del child2.fitness.values

            # Aplicar mutação
            for mutant in offspring:
                if random.random() < self.setup.MUTPB:
                    self.setup.toolbox.mutate(mutant)
                    del mutant.fitness.values

            #  Avaliar o fitness dos novos indivíduos
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = map(self.setup.toolbox.evaluate, invalid_ind)
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = [fit]

            #! Aplicar RCE
            if RCE and ((current_generation + 1) % self.setup.num_repopulation == 0):
                print("")
                self.cout(
                    f"RCE being applied! - Generation = {current_generation + 1} ",
                )
                #! f - copia pop aleatória modificada retornada para pop atual
                new_population = self.aplicar_RCE(
                    current_generation + 1, population[num_pop]
                )
                #print("\nPopulação gerada pelo RCE\n", population[num_pop])
                population[num_pop][:] = new_population
            else:
                population[num_pop][:] = offspring

            # Registrar estatísticas no logbook
            self.elitismoSimples(population[num_pop])
            self.registrarDados(current_generation)
            record = self.stats.compile(population[num_pop])
            self.logbook.record(gen=current_generation, **record)

        # Retornar população final, logbook e elite
        return population[num_pop], self.logbook, self.hof[0]

    def visualizarPopAtual(self, geracaoAtual, stats):

        for i in range(len(self.POPULATION)):
            datasetIndividuals = {
                "Generations": geracaoAtual + 1,
                "index": i,
                "Variaveis de Decisão": self.POPULATION[i],
                "Fitness": self.POPULATION[i].fitness.values,
                "Media": stats[0],
                "Desvio Padrao": stats[1],
                "RCE": " - ",
            }
            self.allIndividualValuesArray.append(datasetIndividuals)

    def cout(self, msg):
        print(
            "\n=========================================================================================================="
        )
        print("\t", msg)
        print(
            "==========================================================================================================\n"
        )

## Exemplo

In [180]:
import time
import json
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import math


# Exemplo dados tubina eolica

X = turbina[['WindSpeed(m/s)']].values
y = turbina[['ActivePower(kW)']].values
data_hora = turbina[['Data/hora']].values
curva = turbina[['Curva_Teórica(KWh)']].values
vento = turbina[['Direção do Vento']].values

print(X.shape,y.shape)


def rastrigin_decisionVariables( individual, decision_variables):
        rastrigin = 10 * len(decision_variables)
        for i in range(len(decision_variables)):
            rastrigin += individual[i] * individual[i] - 10 * (
                math.cos(2 * np.pi * individual[i])
            )
        return rastrigin

turbina = {
    "num_var":5,
    "tipo": int,
    "fitness": rastrigin_decisionVariables
}

#turbina["var"] = [x,y,data_hora, curva, vento]
print(turbina)




(50530, 1) (50530, 1)
{'num_var': 5, 'tipo': <class 'int'>, 'fitness': <function rastrigin_decisionVariables at 0x000001E222EDB880>}


In [181]:
## problema da mochila - combinatria do tipo binario

# SA -> https://github.com/brunodeathayde/SA_TSP/blob/main/main_SA.py

#evolution
prob_dict = {
    "tipo": "binario",
    "num_var": 8,
    "pop": 10,
    "fitness": 0,
}

#problema
multa = 1000
capacidade_mochila = 15
quantidade_itens = [1,8,3,12,7,5,6,3]
lucro = [2,5,3,10,8,1,3,4]
fitness = lambda x: (capacidade_mochila - sum(quantidade_itens[x])) * multa * sum(peso)

#print(fitness(2))

In [197]:
def pesquisaLinear(array,num,delta = 2):
    pos = 0
    count = 0

    while ((pos < len(array)) and (array[pos]) != num):
        pos = pos + 1

    # pegando quantos valores nao sao nulos
    for value in array:
        if value != 0.0:
          count += 1
    print("Valores não nulos = ",count)

    
    # valores a presquisar
    try:
        if (array[pos] == num):
            return True
    except:
        print("nao encontrado")

    # se o contador for maior que delta
    if (count >= delta):
        print("Array diferente")
        return True

    else:
        return False

pesquisaLinear(quantidade_itens,4)

Valores não nulos =  8
nao encontrado
Array diferente


True

In [187]:

def countaValues(values):
  """Conta o número de valores não nulos em um array.

  Args:
    values: Um array de valores.

  Returns:
    A quantidade de valores não nulos no array.
  """

  count = 0
  for value in values:
    if value != 0.0:
      count += 1

  return count

arr = [0.0, 0.0, 0.1979844174315517, 2.054455165215103, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
countaValues(arr)


2

## Main

In [183]:


if __name__ == "__main__":
    tempo1 = time.time()

    # Setup
    params = load_params(r"C:\Users\rpped\Documents\Estagio - Pedro V - Usinagem\Code\assets\parameters.json")
    setup = Setup(params)
    alg = AlgoritimoEvolutivoRCE(setup)
    #data_visual = DataExploration()
    # old_alg = AlgEvolution(setup)

    pop_with_repopulation, logbook_with_repopulation, best_variables = alg.run(
        RCE=True,
        #fitness_function=rastrigin_decisionVariables,
        #decision_variables=(X, y,data_hora,curva,vento),
    )

    print("\n\nEvolução concluída  - 100%")

    # Salvar resultados
    alg.cout("---> RESULTADOS OBTIDOS")
    #data_visual.displayDataFrameInfo(alg.allIndividualValuesArray, "Todos Individuos Gerados")
    # data_visual.show_conjuntoElite(alg.pop_RCE)


    # Visualização dos resultados
    alg.cout("VISUALIZANDO OS RESULTADOS")
    #data_visual.show_rastrigin_benchmark(logbook_with_repopulation,best_variables)
    #data_visual.visualize(
     #   logbook_with_repopulation, pop_with_repopulation, repopulation=True
    #)
    #data_visual.statistics_per_generation_df(logbook_with_repopulation)

    tempo2 = time.time()
    print(f"Total execution time: {round(tempo2-tempo1,2)} seconds")

Método escolhido: Minimizar


C:\Users\rpped\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\rpped\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\rpped\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "




	 RCE being applied! - Generation = 20 

Elitismo HOF Index[0] [-0.03988759805846964, -0.011103074235420785, 1.913930457119333, -1.0419747289062142, 1.085604048893309, -1.0170125083392252, 1.001192151007106, -0.9806074871738781, 0.05582367714203418, -2.043573880971278] 
 Fitness = (17.740941869302663,) | Diversidade = -1.0776089435227039

	 CRITÉRIO 2 RCE 

Fitness calculado com 0.4 = 25
array ind selecionados:  [0.0, 0.0, 0.1979844174315517, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

 Soma Diff =  0.1979844174315517
contador =  0
contador 2 =  0
Não é diferente 
Ind Diferente RCE [0.0, 0.0, 0.1979844174315517, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
array ind selecionados:  [0.0, 0.0, 0.1979844174315517, 2.054455165215103, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

 Soma Diff =  2.2524395826466543
contador =  0
contador 2 =  0
Não é diferente 
Ind Diferente RCE [0.0, 0.0, 0.1979844174315517, 2.054455165215103, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
array ind selecionados:  [0.0, 0.0, 0.1979844174315517, 0.0, 0.0, 0

,Generations,index,Variaveis de Decisão,Fitness,RCE,Diversidade
0,20,0,"[-0.03988759805846964, -0.011103074235420785, ...",17.740942,HOF,-1.077609
1,20,1,"[-0.03988759805846964, -0.011103074235420785, ...",19.483130,SIM,-0.879625
2,20,2,"[-0.03988759805846964, -0.011103074235420785, ...",19.107496,SIM,1.174831
3,20,3,"[2.255529819030726, 1.224620563516348, 2.95598...",161.888829,,-3.804637
4,20,4,"[1.0712344132906484, 0.11315683211597971, 2.74...",179.203525,,-13.177479
5,20,5,"[-3.630264097733959, -2.4907213172781577, -4.3...",218.013805,,-1.601426
6,20,6,"[-2.1068207508179446, -2.180962398465324, -2.9...",187.391659,,-10.974907
7,20,7,"[-4.675133855231512, 1.8564705404083774, -1.52...",199.342215,,1.444844
8,20,8,"[-1.8879683610843303, 1.8449094687233867, 4.09...",131.415055,,-9.468760
9,20,9,"[-1.4521442777864833, 4.578019548523634, -3.60...",181.842753,,-14.120351




	 RCE being applied! - Generation = 40 

Elitismo HOF Index[0] [-0.013653324919438992, -0.011103074235420785, 1.913930457119333, -1.0419747289062142, 0.9397531153784261, -1.0170125083392252, 0.00718170074759783, -0.9806074871738781, 0.05582367714203418, -1.9364498316609742] 
 Fitness = (15.459906687819204,) | Diversidade = -2.0841120048477606

	 CRITÉRIO 2 RCE 

Fitness calculado com 0.4 = 22
array ind selecionados:  [0.0, 0.0, 0.0, 0.0, 0.14585093351488287, 0.0, 0.0, 0.0, 0.0, 0.0]

 Soma Diff =  0.14585093351488287
contador =  0
contador 2 =  0
Não é diferente 
Ind Diferente RCE [0.0, 0.0, 0.0, 0.0, 0.14585093351488287, 0.0, 0.0, 0.0, 0.0, 0.0]
array ind selecionados:  [0.0, 0.0, 0.0, 0.0, 0.14585093351488287, 0.0, 0.0, 0.0, 0.15468313881882734, 0.0]

 Soma Diff =  0.3005340723337102
contador =  0
contador 2 =  0
Não é diferente 
Ind Diferente RCE [0.0, 0.0, 0.0, 0.0, 0.14585093351488287, 0.0, 0.0, 0.0, 0.15468313881882734, 0.0]
array ind selecionados:  [0.02623427313903065, 0.0, 0

...



	 RCE being applied! - Generation = 80 

Elitismo HOF Index[2] [-0.013653324919438992, -0.011103074235420785, -2.035400641798732, -1.0419747289062142, 0.03672369210612292, -1.0170125083392252, 0.00718170074759783, 0.06855057935087672, 0.05582367714203418, -1.025639178473316] 
 Fitness = (9.961507233445126,) | Diversidade = -4.976503807325716

	 CRITÉRIO 2 RCE 

Fitness calculado com 0.4 = 14
array ind selecionados:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08071692308500089]

 Soma Diff =  0.08071692308500089
contador =  0
contador 2 =  0
Não é diferente 
Ind Diferente RCE [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08071692308500089]
Tamanho Elite =  2
Tamanho Elite =  1

	 CRITERIO 3 - População aleatória modificada [HOF,RCE,Aleatorio] 

Individuos da nova população aleatória


,Generations,index,Variaveis de Decisão,Fitness,RCE,Diversidade
0,80,0,"[-0.013653324919438992, -0.011103074235420785,...",9.961507,HOF,-4.976504
1,80,1,"[-0.013653324919438992, -0.011103074235420785,...",12.155076,SIM,-5.057221
2,80,2,"[-2.7790533656861784, 1.7730388188819726, -2.5...",168.630354,,-5.324663
3,80,3,"[2.5263903399143537, -4.949522885036796, 4.367...",201.333456,,7.557259
4,80,4,"[-2.78888430388015, 1.7592288144155699, -4.725...",252.504812,,0.077127
5,80,5,"[-3.979647766822451, 2.7202465398460722, 3.557...",168.911498,,-7.878934
6,80,6,"[-4.885966891892869, -3.723934258183896, -1.60...",225.884021,,-15.107396
7,80,7,"[-0.8652838405161329, -2.188850635124004, 0.37...",173.554826,,-1.817166
8,80,8,"[1.7912557506967843, -2.3679159228854814, -2.4...",171.607324,,1.420570
9,80,9,"[-1.554322373090363, 4.152669011437985, -4.852...",226.782620,,-5.757844




	 RCE being applied! - Generation = 100 

Elitismo HOF Index[2] [-0.013653324919438992, -0.011103074235420785, 0.9049264975343343, -0.024436516194822566, 0.03672369210612292, -1.0170125083392252, 0.00718170074759783, 0.03767878670445235, 0.05582367714203418, -1.0230954357107784] 
 Fitness = (6.142470216434134,) | Diversidade = -1.0469665051651444

	 CRITÉRIO 2 RCE 

Fitness calculado com 0.4 = 9
Nenhum indivíduo atende aos critérios. :( 
Tamanho Elite =  1
Tamanho Elite =  0

	 CRITERIO 3 - População aleatória modificada [HOF,RCE,Aleatorio] 

Individuos da nova população aleatória


,Generations,index,Variaveis de Decisão,Fitness,RCE,Diversidade
0,100,0,"[-0.013653324919438992, -0.011103074235420785,...",6.142470,HOF,-1.046967
1,100,1,"[4.169093237749297, -4.745718456760776, 0.0124...",217.812418,,-1.299065
2,100,2,"[-0.356572192675932, 3.721958705148883, -2.540...",158.423392,,2.568104
3,100,3,"[2.8942066863217475, -2.059497003005151, 1.363...",138.051642,,6.845003
4,100,4,"[2.265457159620569, -0.9005127136074726, 3.535...",156.376932,,0.404585
5,100,5,"[-4.431130745177272, 2.6307872001367114, 4.816...",251.813094,,0.205849
6,100,6,"[-4.761228534973608, 1.5127174139527266, 4.043...",223.207921,,-0.411454
7,100,7,"[1.0824820712452388, 0.10903477556632968, -4.6...",148.396107,,2.023315
8,100,8,"[-1.2666017222554933, 1.041611259637941, -3.01...",205.670729,,-10.095088
9,100,9,"[1.6113097985999785, -5.0708519327601005, 4.45...",254.351283,,11.277259




Evolução concluída  - 100%

	 ---> RESULTADOS OBTIDOS


	 VISUALIZANDO OS RESULTADOS

Total execution time: 3.5 seconds
